In [1]:
import pandas as pd
import numpy as np
import time
import os
import datetime as dt
from functools import reduce

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [2]:
def Betano_file_teams(w1,w2,league): #w1 -> df Sky |||| w2 -> df betano
    #Vamos tornar os nomes das equipas iguais nas duas dfs
    try:
        op = open('BetanoTeams_'+league+'.pickle','rb')
        same = pickle.load(op)
        op.close()
    except:
        print('Não leu ficheiro')
        same = [] # lista com os clubes com o mesmo nome [nome Sky,nome betano]
    # vamos manter o nome do Sky
    try:
        sky_teams =  list(set(list(np.unique(list(w1['HomeTeam'])))+list(np.unique(list(w1['AwayTeam'])))))
        betano_teams = list(set(list(np.unique(list(w2['HomeTeam'])))+list(np.unique(list(w2['AwayTeam'])))))
    except:
        sky_teams =  list(set(list(np.unique(list(w1['HT'])))+list(np.unique(list(w1['AT'])))))
        betano_teams = list(set(list(np.unique(list(w2['HT'])))+list(np.unique(list(w2['AT'])))))
    i = 0
    betano_teams_copy = betano_teams.copy()
    sky_teams_copy = sky_teams.copy()
    if same==[]:
        same=[]
        restos=[]
        for ii in range(len(sky_teams)):
            nn=0
            found=False
            while nn<len(betano_teams) and not(found):
                if sky_teams[ii]==betano_teams[nn]:
                    same+=[[sky_teams[ii],betano_teams[nn]]]
                    del betano_teams[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[sky_teams[ii]]

        if restos==[]:
            op = open('BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            restos_copy=restos.copy()
            betano_teams_copy=betano_teams.copy()
            same_copy=same.copy()
            t=0
            while betano_teams!=[]:
                print('------')
                print(restos)
                ok=input(str(betano_teams[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    betano_teams=betano_teams_copy
                    restos=restos_copy
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,betano_teams[t]],[betano_teams[t],ok]]
                    del betano_teams[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(betano_teams):
                    t=0
            op = open('BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
    else:
        same_sky = [same[i][0] for i in range(len(same))]
        same_betano = [same[i][1] for i in range(len(same))]
        missing_sky = [sky_teams[i] for i in range(len(sky_teams)) if (sky_teams[i] not in same_sky)]
        missing_betano = [betano_teams[i] for i in range(len(betano_teams)) if (betano_teams[i] not in same_betano)]
        missing_sky_copy = missing_sky.copy()
        missing_betano_copy = missing_betano.copy()
        same_copy=same.copy()
        
        restos=[]
        for ii in range(len(missing_sky)):
            nn=0
            found=False
            while nn<len(missing_betano) and not(found):
                if missing_sky[ii]==missing_betano[nn]:
                    same+=[[missing_sky[ii],missing_betano[nn]]]
                    del missing_betano[nn]
                    found=True
                nn+=1

            if not found:
                restos+=[missing_sky[ii]]

        if restos==[]:
            op = open('BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same
        else:
            t=0
            while (restos!=[]) or (missing_betano!=[]):
                if len(restos) != len(missing_betano):
                    print('We got a problemoooo')
                    raise('aa')
                print(restos)
                ok = input(str(missing_betano[t])+'? (write "passa" if not sure Or "reset" to restart) ')
                if ok!='passa' and (ok not in restos) and (ok!='reset'):
                    print('Erro na palavra')
                    continue
                if ok=='passa':
                    t+=1
                    continue
                if ok=='reset':
                    t=0
                    restos=restos_copy.copy()
                    missing_betano=missing_betano_copy.copy()
                    same=same_copy.copy()
                    continue
                else:
                    same+=[[ok,missing_betano[t]],[missing_betano[t],ok]]
                    del missing_betano[t]
                    del restos[np.where(np.array(restos)==ok)[0][0]]
                if t==len(missing_betano):
                    t=0
            op = open('BetanoTeams_'+league+'.pickle','wb')
            pickle.dump(same,op)
            op.close()
            return same

In [ ]:
def Betano_utn(sky,betano,league): #update_team_names
# vamos mudar o nome das dfs de betano para o nome das dfs do sky
#assim podemos fazer um merge usando o np.where
    same = Betano_file_teams(sky,betano,league)
    same_sky = [same[i][0] for i in range(len(same))]
    same_betano = [same[i][1] for i in range(len(same))]
    #HOME
    for i in range(len(same_betano)):
        try:
            replaceH=list(np.where(betano['HT']==same_betano[i])[0])
            home='HT'
        except:
            replaceH=list(np.where(betano['HomeTeam']==same_betano[i])[0])
            home='HomeTeam'
        for n in replaceH:
            betano.at[n,home]=same_sky[i]
    #AWAY
    for i in range(len(same_betano)):
        try:
            replaceH=list(np.where(betano['AT']==same_betano[i])[0])
            away='AT'
        except:
            replaceH=list(np.where(betano['AwayTeam']==same_betano[i])[0])
            away='AwayTeam'
        for n in replaceH:
            betano.at[n,away]=same_sky[i]  
    return sky,betano

In [3]:
def Betano_odds(sky,betano):
    if 'ODDH_Betano' not in list(sky.columns):
        sky['ODDH_Betano']=np.nan
        sky['ODDD_Betano']=np.nan
        sky['ODDA_Betano']=np.nan
    for b in range(len(betano)):
        aa = list(np.where(np.logical_and(np.logical_and(sky['HT']==betano.iloc[b]['HT'],sky['AT']==betano.iloc[b]['AT']),
                                          sky['League']==betano.iloc[b]['League']))[0])
        if aa!=[]:
            if len(aa)>1:
                print('Double Game, Betano_Odds')
                raise('aa')
            else:
                sky.at[aa[0],'ODDH_Betano']=betano.iloc[b]['ODDH']
                sky.at[aa[0],'ODDD_Betano']=betano.iloc[b]['ODDH']
                sky.at[aa[0],'ODDA_Betano']=betano.iloc[b]['ODDH']
    return sky

In [ ]:
def prepare__df(df,season,league,under=True):
    if under:
        print('1',len(df.index))
        df=df.loc[((df["League"]==league))]
        w=[]
        print('2',len(df.index))
        for i in range(len(df.index)):
            w+=[df.iloc[i]['Date']]
        w2=[]
        for n in w: 
            w2+=[UnderStats_Date(n)]
        df=df.drop(['Date'],axis=1)    
        df['Date']=w2   
        df=df.sort_values(['Date'],ascending=True)
        df=df.reset_index(drop=True)
        #last_year=year.df.iloc[-1]['Date']
        found_i=False
        found_f=False
        nn=0
        print(len(df.index))
        while nn <len(df.index) and not(found_i and found_f):
            data=df.iloc[nn]['Date']
            if data.year==(season-1) and data.month>=8 and not(found_i):
                i=nn
                found_i=True
            if data.year==(season) and data.month>=8 and not(found_f):
                f=nn
                found_f=True
            nn+=1
        if not(found_f) and found_i:
            df=df[i:]
        elif found_f and found_i:
            df=df[i:f]
        else:
            raise('WTF!! prepare__df')
        df=df.reset_index(drop=True)
        return df
        
    else:
        print('1',len(df.index))
        df=df.loc[((df["League"]==' '+league))]
        print('2',len(df.index))
        df=df.reset_index(drop=True)
        w=[]
        for i in range(len(df.index)):
            w+=[df.iloc[i]['Date']]
        w2=[]
        w22=[]
        for n in w: 
            #print(SkySports_Date(n)[0].year)
            w2+=[SkySports_Date(n)[0]]
            w22+=[SkySports_Date(n)[1]]
        df=df.drop(['Date'],axis=1)    
        df['Date']=w2 
        df['Time']=w22
        df=df.sort_values(['Date','Time'],ascending=[True,True])
        df=df.reset_index(drop=True)
        print('3',len(df.index))
        #last_year=year.df.iloc[-1]['Date']
        found_i=False
        found_f=False
        nn=0
        while nn <len(df.index) and not(found_i and found_f):
            data=df.iloc[nn]['Date']
            #print(data.year,data.month)
            if data.year==(season-1) and data.month>=8 and not(found_i):
                #print('yellow')
                i=nn
                found_i=True
            if data.year==(season) and data.month>=8 and not(found_f):
                #print('oki oki')
                f=nn
                found_f=True
            nn+=1
        if not(found_f) and found_i:
            df=df[i:]
        elif found_f and found_i:
            df=df[i:f]
        else:
            raise('WTF!! prepare__df')
        df=df.reset_index(drop=True)
        df=all_nan(df)
        return df

In [103]:
def Prepare__Df(df,season,league): 
    print('Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"')
    tydate = np.array(list(map(lambda x:type(x),list(df['Date']))))
    tstamp = list(map(lambda x:x==pd._libs.tslibs.timestamps.Timestamp,list(df['Date'])))
    if len(tstamp)==len(df):
        stamp = True
    
    elif len(np.where(tydate==str)[0])==len(df):
        ok = np.array(list(map(lambda x:(x[4]=='-') and (x[7]=='-'),list(df['Date']))))
        if not (ok.all()):
            print('Man, as datas não estão bem')
            ok2 = list(np.where(list(ok)==False)[0])
            print(str(ok2),'São os índices maus' )
            raise('aa')
        stamp = False
    else:
        print(tstamp)
        print('OOps, temos vários tipos de datas nesta df')
        raise('aa')
    
    # A parte em cima serve só para ver as datas e verificar que podemos fazer o Prepare__Df
    if not stamp: #vamos passar para datetime (o timestamp é basicamente o mesmo do datetime)
        df['Date']=list(map(lambda x:dt.datetime.strptime(x,'%Y-%m-%d'),list(df['Date'])))
    if season==2020: #Corona season
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime(2019,8,4),df['Date']<=dt.datetime(2020,8,3)),df['League']==league))[0])]
    elif season==2021: #After Corona season
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime(2020,8,8),df['Date']<=dt.datetime(2021,8,3)),df['League']==league))[0])]    
    else:
        df1=df.loc[list(np.where(np.logical_and(np.logical_and(df['Date']>=dt.datetime((league-1),8,1),df['Date']<=dt.datetime(league,8,1)),df['League']==league))[0])]
        print('Lembrar de verificar as datas limite')    
    return df1.sort_values(['Date','Time'],ascending=[True,True]).reset_index(drop=True)

In [47]:
df_Betano = pd.read_csv('C://Users//joaom//Documents//Projetos//PYTHON//Apostas//AAPredictions//BetanoOdds//Filtered_Data//Betano0.csv')

In [104]:
Prepare__Df(df_Betano,2020,'Premier_League')

Prepare_Df só funciona com as datas já em formato YYYY-mm-dd, e em ligas "normais"


,Unnamed: 0,League,Date,Time,HT,AT,ODDH,ODDD,ODDA,ODD_25+,ODD_25-,ODD_BthScore_Yes,ODD_BthScore_No,ODD_1X,ODD_12,ODD_X2,Scrape_Date,Scrape_Time
0,28,Premier_League,2020-07-19,1400,Bournemouth,Southampton,2.40,3.65,2.78,1.63,2.02,1.50,2.16,1.40,1.23,1.53,2020-07-19,1145
1,73,Premier_League,2020-07-19,1600,Tottenham,Leicester,2.46,3.53,3.20,1.95,1.7,1.67,1.87,1.35,1.27,1.53,2020-07-19,1145
2,50,Premier_League,2020-07-20,1800,Brighton,Newcastle,1.78,3.45,4.35,1.91,1.65,1.80,1.75,1.18,1.23,1.91,2020-07-19,1145
3,74,Premier_League,2020-07-20,1800,Sheffield United,Everton,2.10,3.20,3.45,2.10,1.53,1.85,1.7,1.27,1.27,1.65,2020-07-19,1145
4,69,Premier_League,2020-07-20,2015,Wolverhampton,Crystal Palace FC,1.50,3.85,6.70,2.12,1.53,2.25,1.45,1.07,1.19,2.45,2020-07-19,1145
5,55,Premier_League,2020-07-21,1800,Watford,Manchester City,7.90,5.10,1.32,1.4,2.4,1.63,1.95,3.10,1.1,1.04,2020-07-19,1145
6,6,Premier_League,2020-07-21,2015,Aston Villa,Arsenal,3.10,3.70,2.05,1.65,2.17,1.50,2.16,1.67,1.21,1.32,2020-07-19,1145
7,63,Premier_League,2020-07-22,1800,Manchester United,West Ham,1.26,5.70,9.00,1.4,2.4,1.75,1.8,1.03,1.12,3.5,2020-07-19,1145
8,2,Premier_League,2020-07-22,2015,Liverpool,Chelsea,2.00,3.45,3.45,1.50,2.13,1.44,2.3,1.27,1.23,1.72,2020-07-19,1145


In [102]:
len(np.where(df_Betano['League']=='Premier_League')[0])

9

In [106]:
len(np.unique(df_Betano['League']))

10